In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
import seaborn as sn
import pandas as pd

In [2]:
dataset = "D:/projects/LARGE Projects/steam_game_analysis/steam_game_final.csv"
df = pd.read_csv(dataset)

In [3]:
df['rating'].dtype

dtype('O')

In [4]:
from sklearn.preprocessing import LabelEncoder

#create instance of label encoder
lab = LabelEncoder()

#perform label encoding on 'team' column
df['date_release'] = lab.fit_transform(df['date_release'])  
df['win'] = lab.fit_transform(df['win'])
df['mac'] = lab.fit_transform(df['mac'])
df['linux'] = lab.fit_transform(df['linux'])
df['rating'] = lab.fit_transform(df['rating'])   

In [5]:
'''
df.win=df.win.astype('category').cat.codes
df.mac=df.mac.astype('category').cat.codes
df.linux=df.linux.astype('category').cat.codes
df.rating=df.rating.astype('category').cat.codes
df.date_release=df.date_release.astype('category').cat.codes
'''


df.date_release=df.date_release.astype(np.float32)
df.price_final=df.price_final.astype(np.float32)
df.win = df.win.astype(np.float32)
df.mac = df.mac.astype(np.float32)
df.linux = df.linux.astype(np.float32)
df.rating = df.rating.astype(np.float32)


In [6]:
df

,date_release,win,mac,linux,rating,price_final,steam_deck
0,11.0,1.0,0.0,0.0,8.0,19.99,True
1,11.0,1.0,0.0,0.0,8.0,9.99,True
2,14.0,1.0,0.0,0.0,6.0,2.99,True
3,16.0,1.0,1.0,1.0,8.0,14.99,True
4,17.0,1.0,0.0,0.0,0.0,14.99,True
...,...,...,...,...,...,...,...
46063,21.0,1.0,0.0,0.0,8.0,0.00,True
46064,21.0,1.0,0.0,0.0,8.0,0.00,True
46065,24.0,1.0,0.0,0.0,2.0,0.00,True
46066,24.0,1.0,0.0,0.0,6.0,0.00,True


In [7]:
df.drop(['steam_deck'],axis=1)

,date_release,win,mac,linux,rating,price_final
0,11.0,1.0,0.0,0.0,8.0,19.99
1,11.0,1.0,0.0,0.0,8.0,9.99
2,14.0,1.0,0.0,0.0,6.0,2.99
3,16.0,1.0,1.0,1.0,8.0,14.99
4,17.0,1.0,0.0,0.0,0.0,14.99
...,...,...,...,...,...,...
46063,21.0,1.0,0.0,0.0,8.0,0.00
46064,21.0,1.0,0.0,0.0,8.0,0.00
46065,24.0,1.0,0.0,0.0,2.0,0.00
46066,24.0,1.0,0.0,0.0,6.0,0.00


In [8]:
#feature column creation for dnn

price_final_feature = tf.feature_column.numeric_column("price_final")
win_feature = tf.feature_column.numeric_column("win")
date_release_feature = tf.feature_column.numeric_column("date_release")
mac_feature = tf.feature_column.numeric_column("mac")
linux_feature = tf.feature_column.numeric_column("linux")
'''
win_feature = tf.feature_column.categorical_column_with_vocabulary_list(
    key='win', vocabulary_list=('WT', 'WF'),
    num_oov_buckets=2)
#win_feature = tf.feature_column.categorical_column_with_vocabulary_list("win")
date_release_feature = tf.feature_column.categorical_column_with_hash_bucket("date_release",hash_bucket_size=27)
mac_feature = tf.feature_column.categorical_column_with_vocabulary_list(
    key='mac', vocabulary_list=('MT', 'MF'),
    num_oov_buckets=2)
linux_feature = tf.feature_column.categorical_column_with_vocabulary_list(
    key='linux', vocabulary_list=('LT', 'LF'),
    num_oov_buckets=2)
'''

'\nwin_feature = tf.feature_column.categorical_column_with_vocabulary_list(\n    key=\'win\', vocabulary_list=(\'WT\', \'WF\'),\n    num_oov_buckets=2)\n#win_feature = tf.feature_column.categorical_column_with_vocabulary_list("win")\ndate_release_feature = tf.feature_column.categorical_column_with_hash_bucket("date_release",hash_bucket_size=27)\nmac_feature = tf.feature_column.categorical_column_with_vocabulary_list(\n    key=\'mac\', vocabulary_list=(\'MT\', \'MF\'),\n    num_oov_buckets=2)\nlinux_feature = tf.feature_column.categorical_column_with_vocabulary_list(\n    key=\'linux\', vocabulary_list=(\'LT\', \'LF\'),\n    num_oov_buckets=2)\n'

In [9]:
feat_columns = [price_final_feature,win_feature,date_release_feature,mac_feature,linux_feature]

In [10]:
X_data = df.drop("rating",axis=1)
labels = df["rating"]

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_data, labels, test_size=0.33, random_state=101)


In [15]:
from keras.models import Sequential
from keras.layers import Dense

model = tf.keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.5),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [16]:
X_train = np.asarray(X_train).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)

X_test = np.asarray(X_test).astype(np.float32)
y_test = np.asarray(y_test).astype(np.float32)

In [17]:
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/100
965/965 [==============================] - 2s 1ms/step - loss: -185395.1875 - accuracy: 0.0375 - val_loss: -729544.7500 - val_accuracy: 0.0381
Epoch 2/100
965/965 [==============================] - 1s 1ms/step - loss: -2600818.7500 - accuracy: 0.0371 - val_loss: -5100653.0000 - val_accuracy: 0.0381
Epoch 3/100
965/965 [==============================] - 1s 1ms/step - loss: -9686350.0000 - accuracy: 0.0371 - val_loss: -14696574.0000 - val_accuracy: 0.0381
Epoch 4/100
965/965 [==============================] - 1s 1ms/step - loss: -22555136.0000 - accuracy: 0.0371 - val_loss: -30359574.0000 - val_accuracy: 0.0381
Epoch 5/100
965/965 [==============================] - 1s 1ms/step - loss: -42023356.0000 - accuracy: 0.0371 - val_loss: -52666292.0000 - val_accuracy: 0.0381
Epoch 6/100
965/965 [==============================] - 1s 1ms/step - loss: -68672544.0000 - accuracy: 0.0371 - val_loss: -82340080.0000 - val_accuracy: 0.0381
Epoch 7/100
965/965 [==============================] 

In [18]:
test_loss, test_acc = model.evaluate(X_test, y_test)

476/476 [==============================] - 0s 592us/step - loss: -136634417152.0000 - accuracy: 0.0381


In [199]:
test_acc

0.03808458894491196